In [2]:
import pandas as pd
import numpy as np

# Grafik kütüphaneleri (ileride lazım olacak)
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [10]:

# 1. Veriyi Oku (Tam Dosya Yolu Kullanıldı)
# Lütfen TAM_DOSYA_YOLU kısmını kopyaladığınız yolla değiştirin.
# Örnek: r'C:\Users\ozlem\OneDrive\Masaüstü\turkiye-enflasyon-analizi\data\tcmb_faiz_oranlari.csv'
TAM_DOSYA_YOLU = r'C:\Users\ozlem\OneDrive\Masaüstü\turkiye-enflasyon-analizi\data\tcmb_faiz_oranlari.csv'
df_faiz = pd.read_csv(TAM_DOSYA_YOLU, sep=';')

# 2. Sütunları Seç ve Yeniden Adlandır
df_faiz = df_faiz.rename(columns={'Tarih': 'Tarih', 'Borç Verme': 'Politika_Faizi'})
df_faiz = df_faiz[['Tarih', 'Politika_Faizi']]

# 3. Veri Tiplerini Düzenle

# Tarih sütununu datetime formatına çevir
df_faiz['Tarih'] = pd.to_datetime(df_faiz['Tarih'], format='%d.%m.%Y', errors='coerce')

# Politika Faizi sütununu sayıya (float) çevir (Metin değerler NaN olacak)
df_faiz['Politika_Faizi'] = pd.to_numeric(df_faiz['Politika_Faizi'], errors='coerce')

# 4. Veriyi Sonlandırma
df_faiz.dropna(subset=['Tarih'], inplace=True)
df_faiz = df_faiz.sort_values(by='Tarih').reset_index(drop=True)

print("✅ Faiz Verisi Başarıyla Yüklendi ve Temizlendi (İlk 5 Satır):")
print(df_faiz.head())
print("\nVeri Tipleri:")
print(df_faiz.info())


✅ Faiz Verisi Başarıyla Yüklendi ve Temizlendi (İlk 5 Satır):
       Tarih  Politika_Faizi
0 2002-02-20            62.0
1 2002-03-14            61.0
2 2002-04-08            58.0
3 2002-04-30            55.0
4 2002-08-05            53.0

Veri Tipleri:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Tarih           127 non-null    datetime64[ns]
 1   Politika_Faizi  92 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.1 KB
None


In [11]:
# Sadece sayısal sütunları 100'e bölerek ondalık hale getiriyoruz
df_enflasyon['Yillik_Degisim'] = df_enflasyon['Yillik_Degisim'] / 100
df_enflasyon['Aylik_Degisim'] = df_enflasyon['Aylik_Degisim'] / 100

# Veri tiplerini float'a (ondalıklı sayı) çeviriyoruz (zaten float olacaklar)
df_enflasyon['Yillik_Degisim'] = df_enflasyon['Yillik_Degisim'].astype(float)
df_enflasyon['Aylik_Degisim'] = df_enflasyon['Aylik_Degisim'].astype(float)


print("✅ Enflasyon Verisi Düzeltildi (İlk 5 Satır):")
print(df_enflasyon.head())
print("\nVeri Tipleri (Kontrol):")
print(df_enflasyon.info())

✅ Enflasyon Verisi Düzeltildi (İlk 5 Satır):
       Tarih  Yillik_Degisim  Aylik_Degisim
0 2005-01-01          0.0924         0.0055
1 2005-02-01          0.0869         0.0002
2 2005-03-01          0.0794         0.0026
3 2005-04-01          0.0818         0.0071
4 2005-05-01          0.0087         0.0092

Veri Tipleri (Kontrol):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Tarih           247 non-null    datetime64[ns]
 1   Yillik_Degisim  247 non-null    float64       
 2   Aylik_Degisim   247 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 5.9 KB
None


In [13]:
# 'Tarih' sütununu baz alarak iki tabloyu birleştiriyoruz.
# 'left' merge kullanıyoruz ki, tüm enflasyon verilerine faiz verileri de eşleşsin.
df_birlesik = pd.merge(df_enflasyon, df_faiz, on='Tarih', how='left')

print("✅ Veriler Birleştirildi (İlk 10 Satır):")
print(df_birlesik.head(10))
print("\nEksik Veri Kontrolü:")
print(df_birlesik.isnull().sum())

✅ Veriler Birleştirildi (İlk 10 Satır):
       Tarih  Yillik_Degisim  Aylik_Degisim  Politika_Faizi
0 2005-01-01             924             55             NaN
1 2005-02-01             869              2             NaN
2 2005-03-01             794             26             NaN
3 2005-04-01             818             71             NaN
4 2005-05-01              87             92             NaN
5 2005-06-01             895              1             NaN
6 2005-07-01             782            -57             NaN
7 2005-08-01             791             85             NaN
8 2005-09-01             799            102             NaN
9 2005-10-01             752            179             NaN

Eksik Veri Kontrolü:
Tarih               0
Yillik_Degisim      0
Aylik_Degisim       0
Politika_Faizi    246
dtype: int64


In [14]:
# Eksik faiz değerlerini, kendisinden önceki son geçerli değerle dolduruyoruz.
df_birlesik['Politika_Faizi'].fillna(method='ffill', inplace=True)

# İlerideki eksik verileri önlemek için, geriye dönük de bir doldurma yapalım.
# Enflasyon tablosunun başlangıcından önce faiz kararları varsa, o faiz oranıyla doldurur.
df_birlesik['Politika_Faizi'].fillna(method='bfill', inplace=True)


print("✅ Eksik Veriler Tamamlandı (Kontrol):")
print(df_birlesik.head(10))
print("\nSon Eksik Veri Kontrolü:")
print(df_birlesik.isnull().sum())

✅ Eksik Veriler Tamamlandı (Kontrol):
       Tarih  Yillik_Degisim  Aylik_Degisim  Politika_Faizi
0 2005-01-01             924             55            16.5
1 2005-02-01             869              2            16.5
2 2005-03-01             794             26            16.5
3 2005-04-01             818             71            16.5
4 2005-05-01              87             92            16.5
5 2005-06-01             895              1            16.5
6 2005-07-01             782            -57            16.5
7 2005-08-01             791             85            16.5
8 2005-09-01             799            102            16.5
9 2005-10-01             752            179            16.5

Son Eksik Veri Kontrolü:
Tarih             0
Yillik_Degisim    0
Aylik_Degisim     0
Politika_Faizi    0
dtype: int64


C:\Users\ozlem\AppData\Local\Temp\ipykernel_18500\1481436705.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_birlesik['Politika_Faizi'].fillna(method='ffill', inplace=True)
C:\Users\ozlem\AppData\Local\Temp\ipykernel_18500\1481436705.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_birlesik['Politika_Faizi'].fillna(method='ffill', inplace=True)
C:\Users\ozlem\AppData\Local\Temp\ipykernel_18500\1481436705.py:6: FutureWarning: A

In [15]:
# Yıllık ve Aylık Değişim değerlerini 100'e bölerek doğru ondalık formatına getiriyoruz
df_birlesik['Yillik_Degisim'] = df_birlesik['Yillik_Degisim'] / 100
df_birlesik['Aylik_Degisim'] = df_birlesik['Aylik_Degisim'] / 100

print("✅ Son Düzeltme Yapıldı (Kontrol):")
print(df_birlesik.head())

✅ Son Düzeltme Yapıldı (Kontrol):
       Tarih  Yillik_Degisim  Aylik_Degisim  Politika_Faizi
0 2005-01-01            9.24           0.55            16.5
1 2005-02-01            8.69           0.02            16.5
2 2005-03-01            7.94           0.26            16.5
3 2005-04-01            8.18           0.71            16.5
4 2005-05-01            0.87           0.92            16.5
